In [ ]:
import os
import torch
import torch.nn as nn
from  torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader, Subset
from torch.utils.data.sampler import SubsetRandomSampler
from sklearn.metrics import classification_report
import numpy as np
import copy

from models import init_params as w_init
from models import TargetNet, AllCNN
from trainer import train, eval, loss_picker, optimizer_picker

## Hyper parameters

In [ ]:
data_name = 'cifar10'
batch_size = 64 

num_epochs = 30
learning_rate = 0.01
loss_mode = 'cross'
optimization = 'sgd'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# print(device)

def seed_torch(seed=2022):
    np.random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_torch()

# Train an original model on cifar10 
## Load data

In [ ]:
def get_dataset(data_name):
    if not data_name in ['mnist','cifar10']:
        raise TypeError('data_name should be a string, including mnist,cifar10. ')
    
    if(data_name == 'mnist'):
        trainset = datasets.MNIST('./data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
        testset = datasets.MNIST('./data', train=False, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

    elif(data_name == 'cifar10'):
        transform = transforms.Compose(
            [transforms.ToTensor(),
             transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        trainset = datasets.CIFAR10(root='./data', train=True,
                                                download=True, transform=transform)
        testset = datasets.CIFAR10(root='./data', train=False,
                                                download=True, transform=transform)
    
    return trainset, testset

def get_dataloader(trainset, testset, batch_size, device):
    train_loader  = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=True)

    return train_loader, test_loader

trainset, testset = get_dataset(data_name)
train_loader, test_loader = get_dataloader(trainset, testset, batch_size, device=device)
# print(train_loader)

## Train and save an original model

In [ ]:
from models import init_params as w_init
from models import TargetNet, AllCNN
from trainer import train, eval, loss_picker, optimizer_picker
import time

num_classes = max(train_loader.dataset.targets) + 1 
model = AllCNN(n_channels=3, num_classes=num_classes, filters_percentage=0.5).to(device)
criterion = loss_picker('cross')
optimizer = optimizer_picker(optimization, model.parameters(), lr=learning_rate, momentum=0.9)
best_acc = 0

start = time.perf_counter()
for epo in range(num_epochs):
    print('EPOCH:{}'.format(epo))
    train(model=model, data_loader=train_loader, criterion=criterion, optimizer=optimizer, loss_mode='cross', device=device)
    _, acc = eval(model=model, data_loader=test_loader, mode='', print_perform=False, device=device)
    print('test acc:{}'.format(acc))

    if acc >= best_acc:
        best_acc = acc
        torch.save(model, './checkpoints/original_best_cifar10.pth')
end = time.perf_counter()

print('Time Consuming:', end-start, 'secs' )

# Retrain an model on cifar10 for remain class 
## Load data

In [ ]:
from torch.utils.data import Subset
import random

def get_dataloader(trainset, testset, batch_size, device):
    train_loader  = DataLoader(dataset=trainset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(dataset=testset, batch_size=batch_size, shuffle=True)

    return train_loader, test_loader


def split_class_data(dataset, forget_class, num_forget):
    forget_index = []
    class_remain_index = []
    remain_index = []
    sum = 0
    for i, (data, target) in enumerate(dataset):
        if target == forget_class and sum < num_forget:
            forget_index.append(i)
            sum += 1
        elif target == forget_class and sum >= num_forget:
            class_remain_index.append(i)
            remain_index.append(i)
            sum += 1
        else:
            remain_index.append(i)
    return forget_index, remain_index, class_remain_index




def get_unlearn_loader(trainset, testset, forget_class, batch_size, num_forget, repair_num_ratio=0.01):
    train_forget_index, train_remain_index, class_remain_index = split_class_data(trainset, forget_class, num_forget=num_forget)
    test_forget_index, test_remain_index, _ = split_class_data(testset, forget_class, num_forget=len(testset))

    train_forget_sampler = SubsetRandomSampler(train_forget_index)#5000
    train_forget_sampler_0_5 =  SubsetRandomSampler(train_forget_index[::2])
    train_forget_sampler_0_2 =  SubsetRandomSampler(train_forget_index[::5])
    train_forget_sampler_0_1 =  SubsetRandomSampler(train_forget_index[::10])
    train_forget_sampler_0_05 =  SubsetRandomSampler(train_forget_index[::20])
    train_forget_sampler_0_01 =  SubsetRandomSampler(train_forget_index[::100])

    train_remain_sampler = SubsetRandomSampler(train_remain_index)#45000

    repair_class_sampler = SubsetRandomSampler(repair_class_index)

    test_forget_sampler = SubsetRandomSampler(test_forget_index)#1000
    test_remain_sampler = SubsetRandomSampler(test_remain_index)#9000

    train_forget_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, sampler=train_forget_sampler)
    train_forget_loader_0_5 = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, sampler=train_forget_sampler_0_5)
    train_forget_loader_0_2 = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, sampler=train_forget_sampler_0_2)
    train_forget_loader_0_1 = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, sampler=train_forget_sampler_0_1)
    train_forget_loader_0_05 = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, sampler=train_forget_sampler_0_05)
    train_forget_loader_0_01 = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, sampler=train_forget_sampler_0_01)

    train_remain_loader = torch.utils.data.DataLoader(dataset=trainset, batch_size=batch_size, sampler=train_remain_sampler)

    test_forget_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size, sampler=test_forget_sampler)
    test_remain_loader = torch.utils.data.DataLoader(dataset=testset, batch_size=batch_size, sampler=test_remain_sampler)

    return train_forget_loader, train_forget_loader_0_5, train_forget_loader_0_2, train_forget_loader_0_1, train_forget_loader_0_05, train_forget_loader_0_01, train_remain_loader, test_forget_loader, test_remain_loader,\
         train_forget_index, train_remain_index, test_forget_index, test_remain_index


forget_class = 4
num_forget = 5000
trainset, testset = get_dataset(data_name)
train_forget_loader, train_forget_loader_0_5, train_forget_loader_0_2, train_forget_loader_0_1, train_forget_loader_0_05,train_forget_loader_0_01, train_remain_loader, test_forget_loader, test_remain_loader, \
     train_forget_index, train_remain_index, test_forget_index, test_remain_index \
     = get_unlearn_loader(trainset, testset, forget_class, batch_size, num_forget)

## Train and save the retrain model

In [ ]:
from models import init_params as w_init
from models import TargetNet, AllCNN
from trainer import train, eval, loss_picker, optimizer_picker
import time

num_classes = max(train_loader.dataset.targets) + 1 #if args.num_classes is None else args.num_classes
model = AllCNN(n_channels=3, num_classes=num_classes, filters_percentage=0.5).to(device)

criterion = loss_picker('cross')
optimizer = optimizer_picker(optimization, model.parameters(), lr=learning_rate, momentum=0.9)

best_acc = 0

start = time.perf_counter()
for epo in range(num_epochs):
    print('EPOCH:{}'.format(epo))
    train(model=model, data_loader=train_remain_loader, criterion=criterion, optimizer=optimizer, loss_mode='cross', device=device)
    _, acc = eval(model=model, data_loader=test_remain_loader, mode='', print_perform=False, device=device)
    print('test acc:{}'.format(acc))

    if acc >= best_acc:
        best_acc = acc
        torch.save(model, './checkpoints/retrain_best_cifar10_nf%d_class%d.pth' % (num_forget, forget_class+1))
end = time.perf_counter()

print('Time Consuming:', end-start, 'secs' )

## Evaluate retrain model

In [ ]:
ori_model = ori_model.to(device)
retrain_model = retrain_model.to(device)

_, all_test_acc = eval(model=ori_model, data_loader=test_loader, mode='', print_perform=True, device=device)
_, remain_test_acc = eval(model=ori_model, data_loader=test_remain_loader, mode='', print_perform=False, device=device)
_, forget_test_acc = eval(model=ori_model, data_loader=test_forget_loader, mode='', print_perform=False, device=device)
_, forget_train_acc = eval(model=ori_model, data_loader=train_forget_loader, mode='', print_perform=False, device=device)
_, forget_remain_acc = eval(model=ori_model, data_loader=train_remain_loader, mode='', print_perform=False, device=device)
print('all test acc:{}, forget acc:{}, remain acc:{}, train forget acc:{}, train remain acc:{}'.format(all_test_acc, forget_test_acc, remain_test_acc, forget_train_acc, forget_remain_acc))

_, all_test_acc = eval(model=retrain_model, data_loader=test_loader, mode='', print_perform=True, device=device)
_, remain_test_acc = eval(model=retrain_model, data_loader=test_remain_loader, mode='', print_perform=False, device=device)
_, forget_test_acc = eval(model=retrain_model, data_loader=test_forget_loader, mode='', print_perform=False, device=device)
_, forget_train_acc = eval(model=retrain_model, data_loader=train_forget_loader, mode='', print_perform=False, device=device)
_, forget_remain_acc = eval(model=retrain_model, data_loader=train_remain_loader, mode='', print_perform=False, device=device)
print('all test acc:{}, forget acc:{}, remain acc:{}, train forget acc:{}, train remain acc:{}'.format(all_test_acc, forget_test_acc, remain_test_acc, forget_train_acc, forget_remain_acc))

## Distance between W(D) and W(D_r)

In [ ]:
def distance(model,model0):
    distance=0
    normalization=0
    for (k, p), (k0, p0) in zip(model.named_parameters(), model0.named_parameters()):
        space='  ' if 'bias' in k else ''
        current_dist=(p.data-p0.data).pow(2).sum().item()
        current_norm=p.data.pow(2).sum().item()
        distance+=current_dist
        normalization+=current_norm
    print(f'Distance: {np.sqrt(distance)}')
    print(f'Normalized Distance: {1.0*np.sqrt(distance/normalization)}')
    return 1.0*np.sqrt(distance/normalization)

pruned_model = torch.load('./checkpoints/pruned_model_cifar10.pth').to(device)

distance(ori_model, retrain_model)
distance(ori_model, pruned_model)
distance(retrain_model, pruned_model)

## Test the Privacy Performance by MIA

In [ ]:
from sklearn.svm import SVC
import tqdm
import torch.nn.functional as F

def entropy(p, dim = -1, keepdim = False):
    return -torch.where(p > 0, p * p.log(), p.new([0.0])).sum(dim=dim, keepdim=keepdim)

def collect_prob(data_loader, model):
    
    # data_loader = torch.utils.data.DataLoader(data_loader.dataset, batch_size=1, shuffle=False)
    prob = []
    with torch.no_grad():
        for idx, batch in enumerate(tqdm.tqdm(data_loader, leave=False)):
            batch = [tensor.to(next(model.parameters()).device) for tensor in batch]
            data, target = batch
            output = model(data)
            prob.append(F.softmax(output, dim=-1).data)#F.softmax(output, dim=-1).data)
    return torch.cat(prob)

def get_membership_attack_data(retain_loader, forget_loader, test_loader, model):    
    retain_prob = collect_prob(retain_loader, model)#[45000, 10]
    forget_prob = collect_prob(forget_loader, model)#[5000, 10]
    test_prob = collect_prob(test_loader, model)#[10000, 10]
    
    X_r = torch.cat([retain_prob, test_prob]).cpu().numpy().reshape(-1, 10)
    Y_r = np.concatenate([np.ones(retain_prob.size(0)), np.zeros(test_prob.size(0))])
    
    X_f = (forget_prob).cpu().numpy().reshape(-1, 10)
    Y_f = np.concatenate([np.ones(forget_prob.size(0))])    
    return X_f, Y_f, X_r, Y_r


def get_membership_attack_prob(retain_loader, forget_loader, test_loader, model):
    
    X_f, Y_f, X_r, Y_r = get_membership_attack_data(retain_loader, forget_loader, test_loader, model)
    clf = SVC(C=2,gamma='auto',kernel='rbf')
    #clf = LogisticRegression(class_weight='balanced',solver='lbfgs',multi_class='multinomial')
    clf.fit(X_r, Y_r)
    results = clf.predict(X_f)#???
    return results.mean()

def get_entropy(retain_loader,forget_loader,test_loader, model):
    retain_prob = collect_prob(retain_loader, model)#.cpu().numpy()#[45000, 10]
    forget_prob = collect_prob(forget_loader, model)#.cpu().numpy()#[5000, 10]
    test_prob = collect_prob(test_loader, model)#.cpu().numpy()#[10000, 10]
    retain_entropy = entropy(retain_prob)
    forget_entropy = entropy(forget_prob)
    test_entropy = entropy(test_prob)
    return retain_entropy, forget_entropy, test_entropy

def plot_entropy_dist(model, ax, title):
    train_loader_full, test_loader_full = datasets.get_loaders(dataset, batch_size=100, seed=0, augment=False, shuffle=False)
    indexes = np.flatnonzero(np.array(train_loader_full.dataset.targets) == class_to_forget)
    replaced = np.random.RandomState(0).choice(indexes, size=100 if num_to_forget==100 else len(indexes), replace=False)
    X_f, Y_f, X_r, Y_r = get_membership_attack_data(train_loader_full, test_loader_full, model, replaced)
    sns.distplot(np.log(X_r[Y_r==1]).reshape(-1), kde=False, norm_hist=True, rug=False, label='retain', ax=ax)
    sns.distplot(np.log(X_r[Y_r==0]).reshape(-1), kde=False, norm_hist=True, rug=False, label='test', ax=ax)
    sns.distplot(np.log(X_f).reshape(-1), kde=False, norm_hist=True, rug=False, label='forget', ax=ax)
    ax.legend(prop={'size': 14})
    ax.tick_params(labelsize=12)
    ax.set_title(title,size=18)
    ax.set_xlabel('Log of Entropy',size=14)
    ax.set_ylim(0,0.4)
    ax.set_xlim(-35,2)

def membership_attack(retain_loader,forget_loader,test_loader,model,model_name):
    prob = get_membership_attack_prob(retain_loader,forget_loader,test_loader,model)
    print("Attack prob for %s: %f" % (model_name, prob))
    return prob
# ori_model = torch.load('./checkpoints/original_best_cifar10.pth', map_location=torch.device('cpu')).to(device)
# prob_original = membership_attack(train_remain_loader, train_forget_loader, test_loader, ori_model, model_name='original model') 
# prob = membership_attack(train_remain_loader, train_forget_loader, test_loader, pruned_model, model_name='pruned_model')

## Boundary Shrink

In [ ]:
from adv_generator import LinfPGD, inf_generator, FGSM
import tqdm
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from sklearn.metrics import classification_report
import torch.nn.functional as F

norm = True#None
bound = 2.0 
step =  2/255
iter = 10
random_start= False # False if attack != "pgd" else True
poison_epoch = 5

forget_acc_list = []
remain_acc_list = []
train_forget_acc_list = []
train_remain_acc_list = []
time_list = []
mia_list = []

for _ in range(3):
    test_model = copy.deepcopy(ori_model).to(device)
    unlearn_model = copy.deepcopy(ori_model).to(device)

    adv = LinfPGD(test_model, bound, step, iter, norm, random_start, device)
    # adv = FGSM(test_model, bound, norm, random_start, device)
    forget_data_gen = inf_generator(train_forget_loader) 
    batches_per_epoch = len(train_forget_loader)

    criterion = loss_picker('cross')
    optimizer = optimizer_picker(optimization, unlearn_model.parameters(), lr=0.00001, momentum=0.9)

    num_hits = 0
    num_sum = 0
    nearest_label = []

    start = time.perf_counter()
    for itr in tqdm.tqdm(range(poison_epoch * batches_per_epoch)) :

        x, y = forget_data_gen.__next__()
        x = x.to(device)
        y = y.to(device)
        ori_logits = test_model(x)
        test_model.eval()
        x_adv = adv.perturb(x, y, target_y=None, model=test_model)
        adv_logits = test_model(x_adv)
        pred_label = torch.argmax(adv_logits, dim=1)
        if itr >= (poison_epoch - 1)*batches_per_epoch :
            nearest_label.append(pred_label.tolist())
        num_hits += (y != pred_label).float().sum()
        num_sum +=  y.shape[0]

        unlearn_model.train()
        unlearn_model.zero_grad()
        optimizer.zero_grad()

        ori_logits = unlearn_model(x)
        ori_loss = criterion(ori_logits, pred_label)
        loss = ori_loss
        loss.backward()
        optimizer.step()
    end = time.perf_counter()
    tm = end-start
    print('Time Consuming:', tm, 'secs' )

    print('attack success ratio:', (num_hits/num_sum).float() )

    _, test_acc = eval(model=unlearn_model, data_loader=test_loader, mode='', print_perform=False, device=device)
    _, forget_acc = eval(model=unlearn_model, data_loader=test_forget_loader, mode='', print_perform=False, device=device)
    _, remain_acc = eval(model=unlearn_model, data_loader=test_remain_loader, mode='', print_perform=False, device=device)
    _, train_forget_acc = eval(model=unlearn_model, data_loader=train_forget_loader, mode='', print_perform=False, device=device)
    _, train_remain_acc = eval(model=unlearn_model, data_loader=train_remain_loader, mode='', print_perform=False, device=device)
    prob = membership_attack(train_remain_loader, train_forget_loader, test_loader, unlearn_model, model_name='unlearn_model')
    print('test acc:{}, forget acc:{}, remain acc:{}, train forget acc:{}, train remain acc:{}, ASR:{}'.format(test_acc, forget_acc, remain_acc, train_forget_acc, train_remain_acc, prob))
    forget_acc_list.append(forget_acc.cpu().detach().numpy())
    remain_acc_list.append(remain_acc.cpu().detach().numpy())
    train_forget_acc_list.append(train_forget_acc.cpu().detach().numpy())
    train_remain_acc_list.append(train_remain_acc.cpu().detach().numpy())
    time_list.append(tm)
    mia_list.append(prob)

print('######################################################################################################')
print('test forget acc:{}({}), test remain acc:{}({}), train forget acc:{}({}), train remain acc:{}({}), time:{}({}),  ASR:{}({})'.format(np.mean(forget_acc_list), np.var(forget_acc_list), np.mean(remain_acc_list), np.var(remain_acc_list), \
    np.mean(train_forget_acc_list), np.var(train_forget_acc_list), np.mean(train_remain_acc_list), np.var(train_remain_acc_list), np.mean(time_list), np.var(time_list), np.mean(mia_list), np.var(mia_list)))

## Test the Utility Performance


In [ ]:
unlearn_model = torch.load('./checkpoints/advPoisoning_cifar10_nf%d_class%d.pth'%(num_forget, forget_class))
_, test_acc = eval(model=unlearn_model, data_loader=test_loader, mode='', print_perform=True, device=device)
_, forget_acc = eval(model=unlearn_model, data_loader=test_forget_loader, mode='', print_perform=False, device=device)
_, remain_acc = eval(model=unlearn_model, data_loader=test_remain_loader, mode='', print_perform=False, device=device)
_, train_forget_acc = eval(model=unlearn_model, data_loader=train_forget_loader, mode='', print_perform=False, device=device)
_, train_remain_acc = eval(model=unlearn_model, data_loader=train_remain_loader, mode='', print_perform=False, device=device)
print('test acc:{}, forget acc:{}, remain acc:{}, train forget acc:{}, train remain acc:{} '.format(test_acc, forget_acc, remain_acc, train_forget_acc, train_remain_acc))

## Boundary Expanding

In [ ]:
from models import init_params as w_init
from adv_generator import  inf_generator
import tqdm

n_filter2 = int(192*0.5)
num_classes = 10
narrow_model = torch.load('./checkpoints/advPoisoning_cifar10_nf%d_class%d.pth'%(num_forget, forget_class))
feature_extrator = narrow_model.features
classifier = narrow_model.classifier

widen_classifier = nn.Linear(n_filter2, num_classes+1)
w_init(widen_classifier)
widen_model = nn.Sequential(feature_extrator, widen_classifier)
widen_model = widen_model.to(device)

discriminator = nn.Linear(num_classes+1, 2)
w_init(discriminator)
adv_model = nn.Sequential(widen_model, discriminator)
adv_model = adv_model.to(device)

dict = widen_classifier.state_dict()

for name, params in classifier.named_parameters():
    # print(name, params.data)
    if 'weight' in name:
        widen_classifier.state_dict()['weight'][0:10,] = classifier.state_dict()[name][:,]
    elif 'bias' in name:
        widen_classifier.state_dict()['bias'][0:10,] = classifier.state_dict()[name][:,]

forget_data_gen = inf_generator(train_forget_loader) 
batches_per_epoch = len(train_forget_loader)
finetune_epochs = 10

criterion = loss_picker('cross')
optimizer = optimizer_picker(optimization, widen_model.parameters(), lr=0.00001, momentum=0.9)
centr_optimizer = optimizer_picker(optimization, widen_model.parameters(), lr=0.00001, momentum=0.9)
adv_optimizer = optimizer_picker(optimization, adv_model.parameters(), lr=0.001, momentum=0.9)

num_hits = 0
num_sum = 0

start = time.perf_counter()
for itr in tqdm.tqdm(range(finetune_epochs * batches_per_epoch)) :

    x, y = forget_data_gen.__next__()
    x = x.to(device)
    y = y.to(device)

    widen_logits = widen_model(x)
    pred_label = torch.argmax(widen_logits, dim=1)
    #target label
    target_label = torch.ones_like(y,device=device)
    target_label *= num_classes
    
    widen_model.train()
    widen_model.zero_grad()
    optimizer.zero_grad()

    widen_loss = criterion(widen_logits, target_label)
    widen_loss.backward()
    optimizer.step()

pruned_classifier = nn.Linear(n_filter2, num_classes)
for name, params in widen_model[1].named_parameters():
    # print(name)
    if 'weight' in name:
        pruned_classifier.state_dict()['weight'][:,] = widen_model[1].state_dict()[name][0:10,]
    elif 'bias' in name:
        pruned_classifier.state_dict()['bias'][:,] = widen_model[1].state_dict()[name][0:10,]

pruned_model = nn.Sequential(feature_extrator, pruned_classifier)
pruned_model = pruned_model.to(device)
end = time.perf_counter()
print('Time Consuming:', end-start, 'secs' )

# torch.save(widen_model, './checkpoints/widen_model_cifar10_nf%d_class%d.pth' % (num_forget, forget_class))
# torch.save(pruned_model, './checkpoints/pruned_model_cifar10_nf%d_class%d.pth' % (num_forget, forget_class))

In [ ]:
widen_model = torch.load('./checkpoints/widen_model_cifar10.pth')
_, all_test_acc = eval(model=widen_model, data_loader=test_loader, mode='pruned', print_perform=True, device=device)
_, remain_test_acc = eval(model=widen_model, data_loader=test_remain_loader, mode='pruned', print_perform=True, device=device)
_, forget_test_acc = eval(model=widen_model, data_loader=test_forget_loader, mode='pruned', print_perform=True, device=device)
_, forget_train_acc = eval(model=widen_model, data_loader=train_forget_loader, mode='', print_perform=True, device=device)
print('all test acc:{}, forget acc:{}, remain acc:{}'.format(all_test_acc, forget_test_acc, remain_test_acc, forget_train_acc))

In [ ]:
# pruned_model = torch.load('./checkpoints/pruned_model_cifar10_nf%d_class%d.pth' % (num_forget, forget_class))
_, all_test_acc = eval(model=pruned_model, data_loader=test_loader, mode='pruned', print_perform=True, device=device)
_, remain_test_acc = eval(model=pruned_model, data_loader=test_remain_loader, mode='widen', print_perform=True, device=device)
_, forget_test_acc = eval(model=pruned_model, data_loader=test_forget_loader, mode='widen', print_perform=True, device=device)
_, forget_train_acc = eval(model=pruned_model, data_loader=train_forget_loader, mode='widen', print_perform=True, device=device)
_, remain_train_acc = eval(model=pruned_model, data_loader=train_remain_loader, mode='widen', print_perform=False, device=device)
print('all test acc:{}, forget acc:{}, remain acc:{}, forget trained acc:{}, remain trained acc{} '.format(all_test_acc, forget_test_acc, remain_test_acc, forget_train_acc, remain_train_acc))